# Introduction

We are now moving to the final part of the workshop, which involves formulating business recommendations. Our tasks are:
- Determining a global betting odds,
- Dividing the dataset into categories: A, B, C, D, where A is the best group and D is the weakest group,
- Determining the risk of odds based on accepted parameters for each category.

As the last task, in a discussion format, we must consider the fact that we are a new betting company. When formulating our recommendations, we need to identify the risks that may affect our operations. We will perform this task together in a brainstorming session.

# Notebook Configuration

## Import necessary libraries

In [6]:
import pandas as pd

## Loading data into the workspace

> Remember to correctly specify the column separator

In [8]:
df = pd.read_csv('C:\\Users\\alneu\\OneDrive\\Desktop\\processed\\hockey_teams.csv', sep=";")

### Checking data loading accuracy

In [10]:
df

team  season  victories  defeats  overtime_defeats  \
0          Boston Bruins    1990         44       24                 0   
1         Buffalo Sabres    1990         31       30                 0   
2         Calgary Flames    1990         46       26                 0   
3     Chicago Blackhawks    1990         49       23                 0   
4      Detroit Red Wings    1990         34       38                 0   
..                   ...     ...        ...      ...               ...   
602  Tampa Bay Lightning    2011         38       36                 8   
603  Toronto Maple Leafs    2011         35       37                10   
604    Vancouver Canucks    2011         51       22                 9   
605  Washington Capitals    2011         42       32                 8   
606        Winnipeg Jets    2011         37       35                10   

     victory_percentage  scored_goals  received_goals  goal_difference  \
0                 0.550           299             264               35   
1                 0.388           292             278               14   
2                 0.575           344             263               81   
3                 0.613           284             211               73   
4                 0.425           273             298              -25   
..                  ...           ...             ...              ...   
602               0.463           235             281              -46   
603               0.427           231             264              -33   
604               0.622           249             198               51   
605               0.512           222             230               -8   
606               0.451           225             246              -21   

     goals_ratio  
0       1.132576  
1       1.050360  
2       1.307985  
3       1.345972  
4       0.916107  
..           ...  
602     0.836299  
603     0.875000  
604     1.257576  
605     0.965217  
606     0.914634  

[607 rows x 10 columns]

# Determining Betting Odds

Let's review the content of the page: [click](https://trustbet.pl/kursy-bukmacherskie/), where information about methods for determining betting odds can be found. First, we will determine a global odd, which will be the starting point for our analysis (the so-called _baseline scenario_). At this point, we ignore the margin and assume that we are calculating the decimal odd.

Here is the list of steps to be performed to obtain the desired value:
- we will complete the definition of the `get_betting_odds` function, which will take `probability` of a given event as a parameter. We will use it multiple times, so it is worth preparing its implementation now
- then we need to appropriately aggregate the set and determine the **global** probability of the team's victory.

## Implementations of the `get_betting_odds` function

In [12]:
def get_betting_odds(probability):
    return 1/probability

### Some tests to check the correctness of the implementation

In [14]:
def test_get_betting_odds():
    assert get_betting_odds(1) == 1, "Expected 1"
    assert get_betting_odds(0.5) == 2, "Expected 2"
    assert get_betting_odds(0.25) == 4, "Expected 4"
    assert get_betting_odds(0.1) == 10, "Expected 10"
    try:
        get_betting_odds(0)
    except ZeroDivisionError:
        pass
    else:
        assert False, "Expected ZeroDivisionError"

    print("All tests passed!")

test_get_betting_odds()

All tests passed!


### Determining the global odds

Here, determine the probability of any team winning

In [16]:
win_and_loss = df['victories'].sum() + df['defeats'].sum()

win_prob = df['victories'].sum() / win_and_loss
win_prob

0.5317433897660262

In [18]:
win_prob_round = round(win_prob, 4)
win_prob_round

0.5317

Set the global rate here using the `get_betting_odds` function. Round the result to two decimal places.

In [20]:
odds = get_betting_odds(win_prob_round)
print(f"Sázkový kurz:", odds)

Sázkový kurz: 1.880759826970096


In [22]:
odds_round = round(odds, 2)
odds_round

1.88

# Team Categorization

Let's discuss how we can classify teams into _leagues_. We want to establish 4 leagues:
- A - league consisting of the best teams,
- B - league consisting of good teams,
- C - league consisting of average teams,
- D - league consisting of the weakest teams.

The above terms are quite subjective, so for the purpose of this exercise, we will adopt the following assumptions:
- A - the top 5% of teams,
- B - teams performing better than 70% of the group but worse than league A,
- C - teams performing better than 20% of the group but worse than league B,
- D - the remaining teams.

To accomplish this task, we will additionally implement the function `assign_team_to_league`.

> Note: This task looks unassuming, but it is difficult. Remember that during the class, you have access to the instructor, and later to a mentor.

In [24]:
df_avg = df.groupby('team')['victory_percentage'].mean().sort_values(ascending=False).reset_index()
print(df_avg)

                       team  victory_percentage
0         Detroit Red Wings            0.578682
1         New Jersey Devils            0.528227
2             Anaheim Ducks            0.522333
3              Dallas Stars            0.516889
4        Colorado Avalanche            0.516062
5       Pittsburgh Penguins            0.499409
6       Philadelphia Flyers            0.493091
7           St. Louis Blues            0.487364
8             Boston Bruins            0.486217
9       Washington Capitals            0.476500
10        Vancouver Canucks            0.474636
11      Nashville Predators            0.471769
12         New York Rangers            0.468091
13           Buffalo Sabres            0.467565
14       Montreal Canadiens            0.463091
15       Chicago Blackhawks            0.461000
16           Calgary Flames            0.455913
17          Phoenix Coyotes            0.452800
18           Minnesota Wild            0.449000
19      Carolina Hurricanes            0

## Determination of cutoff points for individual leagues

In [26]:
lg_vp = df_avg.describe([0, 0.2, 0.7, 0.95, 1])
lg_vp

victory_percentage
count           35.000000
mean             0.448982
std              0.053134
min              0.366625
0%               0.366625
20%              0.383628
50%              0.452800
70%              0.474063
95%              0.524102
100%             0.578682
max              0.578682

## Determination of odds per league

Here we set the betting odds for each league, which will allow us to draw final conclusions and establish the basic odds for individual teams.

> Remember: After generating the results, it is worth checking if they are reasonable.

In [28]:
percentiles = [0, 0.2, 0.7, 0.95, 1.0]
categories = ['D', 'C', 'B', 'A']

bins = df_avg['victory_percentage'].quantile(percentiles).tolist()

df_avg['group'] = pd.cut(df_avg['victory_percentage'], bins=bins, labels=categories, include_lowest=True)

print(df_avg)

                       team  victory_percentage group
0         Detroit Red Wings            0.578682     A
1         New Jersey Devils            0.528227     A
2             Anaheim Ducks            0.522333     B
3              Dallas Stars            0.516889     B
4        Colorado Avalanche            0.516062     B
5       Pittsburgh Penguins            0.499409     B
6       Philadelphia Flyers            0.493091     B
7           St. Louis Blues            0.487364     B
8             Boston Bruins            0.486217     B
9       Washington Capitals            0.476500     B
10        Vancouver Canucks            0.474636     B
11      Nashville Predators            0.471769     C
12         New York Rangers            0.468091     C
13           Buffalo Sabres            0.467565     C
14       Montreal Canadiens            0.463091     C
15       Chicago Blackhawks            0.461000     C
16           Calgary Flames            0.455913     C
17          Phoenix Coyotes 

In [30]:
odds_dict = {'A': 1.25, 'B': 1.6, 'C': 2.0, 'D': 2.5}

df_avg['odds'] = df_avg['group'].map(odds_dict)

print(df_avg)


                       team  victory_percentage group  odds
0         Detroit Red Wings            0.578682     A  1.25
1         New Jersey Devils            0.528227     A  1.25
2             Anaheim Ducks            0.522333     B  1.60
3              Dallas Stars            0.516889     B  1.60
4        Colorado Avalanche            0.516062     B  1.60
5       Pittsburgh Penguins            0.499409     B  1.60
6       Philadelphia Flyers            0.493091     B  1.60
7           St. Louis Blues            0.487364     B  1.60
8             Boston Bruins            0.486217     B  1.60
9       Washington Capitals            0.476500     B  1.60
10        Vancouver Canucks            0.474636     B  1.60
11      Nashville Predators            0.471769     C  2.00
12         New York Rangers            0.468091     C  2.00
13           Buffalo Sabres            0.467565     C  2.00
14       Montreal Canadiens            0.463091     C  2.00
15       Chicago Blackhawks            0

# Discussion

We have obtained certain odds values for each league. But how does this translate into real business? The entire task was about determining certain values from which a bookmaker can begin operations. Correct determination of these values is critical to attract customers to place bets with us, and on the other hand, inappropriate determination may lead to financial losses in the first days of operation.

For this reason, before translating the results and recommendations into business objectives, the analysis is subjected to discussion. Therefore, we will now take on a review role and would like to verify the steps. To that end, we will collectively discuss and critique our work by answering the following questions together:
- What elements of the analysis were simplified? What was omitted in the analysis?
- Are there any inconsistencies in the estimated odds? What are they?
- How can we improve the odds estimates?
- How can we enrich our initial dataset to make the estimates more accurate and less risky?
- How can we simulate the outcomes of our analysis to verify that they do not lead to financial losses?

This is a discussion panel, and every idea is valuable here.
